In [7]:
# !pip install transformers datasets

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW
from torch.utils.data import DataLoader

In [28]:
sentiment = load_dataset("yelp_polarity")
tr = sentiment['train']
ts = sentiment['test']
tr = tr.train_test_split(test_size=0.97)['train']
ts = ts.train_test_split(test_size=0.97)['train']

In [29]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
batch_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
def tokenize_function(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512, return_tensors="pt")

In [32]:
tr = tr.map(tokenize_function, batched=True)
tr.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|███████████████████████| 16800/16800 [00:07<00:00, 2344.98 examples/s]


In [33]:
train_loader = DataLoader(tr, batch_size=batch_size, shuffle=True)

In [34]:
ts = ts.map(tokenize_function, batched=True)
ts.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
ts_loader = DataLoader(ts, batch_size=batch_size, shuffle=False)

Map: 100%|█████████████████████████| 1140/1140 [00:00<00:00, 2282.69 examples/s]


In [35]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [36]:
for epoch in range(1): 
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

KeyboardInterrupt: 